In [1]:
import sys
import os.path
import numpy as np
import tensorflow as tf
import keras
import xarray as xr
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import ConvLSTM2D, BatchNormalization, MaxPooling3D, TimeDistributed, Flatten, Dense, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint

from parflow_nn.losses import log_loss, rmse, var_loss, var_ratio, metrics
from parflow_nn.preprocess_PF import create_feature_or_target_da
from parflow_nn.write_nc import generate_nc_files
from parflow_nn.config import Config
from cond_rnn import ConditionalRNN

Using TensorFlow backend.


In [2]:
c = Config('parflownn', 'parflow_nn/config.ini')
run_dir = 'washita_clm'
is_clm = True
out_dir = '/glade/scratch/hoangtran/ParFlow-NN/nc_files'

In [3]:
if 1:
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

    run_name = os.path.basename(run_dir)
    static_file = os.path.join(out_dir, f'{run_name}_static.nc')
    forcing_file = os.path.join(out_dir, f'{run_name}_forcings.nc')
    prev_press_file = os.path.join(out_dir, f'{run_name}_prev_press.nc')
    target_satur_file = os.path.join(out_dir, f'{run_name}_satur.nc')
    target_press_file = os.path.join(out_dir, f'{run_name}_press.nc')

    if is_clm:
        target_clm_file = os.path.join(out_dir, f'{run_name}_clm.nc')


In [4]:
if 1:
    # Forcing data
    forcing_input = xr.open_dataset(forcing_file)

    forcing_feature_da, forcing_feature_names = create_feature_or_target_da(
        forcing_input,
        ['forcings'],
        0,
        'feature',
        flx_same_dt=True
    )

    # Add channel dimension
    if is_clm:
        forcing_feature_da = forcing_feature_da.data[:]
        forcing_feature_da = np.swapaxes(forcing_feature_da, 1, 2)
        forcing_feature_da = np.swapaxes(forcing_feature_da, 2, 3)
        forcing_feature_da = np.repeat(forcing_feature_da,
                               repeats=[2] + [1] * (forcing_feature_da.shape[0] - 1),
                               axis=0)  # duplicate the first row
        forcing_feature_da = forcing_feature_da[np.newaxis, ...]
    else:
        forcing_feature_da = forcing_feature_da.data[:, 0, :, :]
        forcing_feature_da = forcing_feature_da[..., np.newaxis]
        forcing_feature_da = forcing_feature_da[np.newaxis, ...]


In [5]:
if 1:
    # Static inputs
    static_input_xr = xr.open_dataset(static_file)

    static_feature_da, static_feature_names = create_feature_or_target_da(
        static_input_xr,
        ['slope_x', 'slope_y', 'perm', 'poros', 'rel_perm_alpha', 'rel_perm_N', 'satur_alpha', 'satur_N',
            'satur_sres', 'satur_ssat', 'tensor_x', 'tensor_y', 'tensor_z', 'spec_storage', 'mannings'],
        0,
        'feature',
        flx_same_dt=True
    )

    # Reduce input
    one_layer_feats = ['slope_x', 'slope_y', 'spec_storage', 'mannings', 'tensor_x', 'tensor_y', 'tensor_z']
    new_static_feature_da = []
    new_static_names = []
    for ii, fname in enumerate(static_feature_names.data):
        if fname.split('_lev')[0] in one_layer_feats:
            if int(fname[-2:]) == 0:
                new_static_feature_da.append(static_feature_da[:, ii, :, :])
                new_static_names.append(fname)
            else:
                continue
        else:
            new_static_feature_da.append(static_feature_da[:, ii, :, :])
            new_static_names.append(fname)

    new_static_feature_da = np.stack(new_static_feature_da, axis=0)
    new_static_feature_da = np.swapaxes(new_static_feature_da, 0, 1)
    new_static_feature_da = np.swaFilters2paxes(new_static_feature_da, 1, 2)
    new_static_feature_da = np.swapaxes(new_static_feature_da, 2, 3)
    new_static_feature_da = np.tile(new_static_feature_da, (forcing_feature_da.shape[1], 1, 1, 1))
    new_static_feature_da = new_static_feature_da[np.newaxis, ...]


In [6]:
if 1:
    # Previous pressure level
    prev_press_input = xr.open_dataset(prev_press_file)
    prev_press_feature_da, prev_press_feature_names = create_feature_or_target_da(
        prev_press_input,
        ['prev_press'],
        0,
        'feature',
        flx_same_dt=True
    )
    prev_press_feature_da = np.swapaxes(prev_press_feature_da.data, 1, 2)
    prev_press_feature_da = np.swapaxes(prev_press_feature_da, 2, 3)
    prev_press_feature_da = prev_press_feature_da[np.newaxis, ...]


In [7]:
if 1:
    target_press_input_xr = xr.open_dataset(target_press_file)
    target_satur_input_xr = xr.open_dataset(target_satur_file)
    if is_clm:
        target_clm_input_xr = xr.open_dataset(target_clm_file)
        target_clm = np.repeat(target_clm_input_xr.clm,
                               repeats=[2]+[1]*(target_clm_input_xr.clm.shape[0] - 1),
                               axis=0) #duplicate the first row
        target_da = np.concatenate([target_press_input_xr.press,
                            target_satur_input_xr.satur,
                            target_clm], axis = 1)
        target_da = target_da[np.newaxis, ...]
    else:
        target_dataset = target_press_input_xr.merge(target_satur_input_xr)
        target_da, target_names = create_feature_or_target_da(
            target_dataset,
            ['press', 'satur'],
            0,
            'target',
            1,
            flx_same_dt=True
        )

        target_da = target_da.data[np.newaxis, ...]

    target_da = np.swapaxes(target_da, 2, 3)
    target_da = np.swapaxes(target_da, 3, 4)


In [8]:
#Conditional RNN
n_sample, n_timestep, nlat, nlon, n_forc_feat = forcing_feature_da.shape
con_forcing = np.reshape(forcing_feature_da, (n_sample, n_timestep, -1))
con_stats = []
con_stat_names = []
for var in static_input_xr.data_vars:
    tmp_stat = static_input_xr[var].data
    if var in ['slope_x', 'slope_y', 'spec_storage', 'mannings', 'tensor_x', 'tensor_y', 'tensor_z']:
        tmp_stat = tmp_stat[:, -1, :, :]
        tmp_stat = tmp_stat[np.newaxis, ...]
        tmp_stat = np.swapaxes(tmp_stat, 0, 1)
    con_stat_names.append(var)
    con_stats.append(tmp_stat)

In [10]:
if 1:
    kernel = 5
    max_num_conditions = 50
    multi_cond_to_init_state_Conv = []
    for i in range(max_num_conditions):
        multi_cond_to_init_state_Conv.append(tf.keras.layers.Conv2D(filters = 3, kernel_size= kernel,
                                                                        padding = "same",
                                                                         data_format="channels_first"))
    multi_cond_p = tf.keras.layers.Conv3D(filters = 1, kernel_size = kernel,
                                               padding = "same",
                                               data_format="channels_last")


In [11]:
if 1:
    init_state_list = []
    for ii, c in enumerate(con_stats):
        init_state_list.append(multi_cond_to_init_state_Conv[ii](c))
    


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [12]:
tf.stack(init_state_list, axis=-1).shape

TensorShape([1, 3, 41, 41, 16])

In [79]:
multi_cond_state = multi_cond_p(tf.stack(init_state_list, axis=-1))
multi_cond_state = tf.reshape(multi_cond_state, [1, -1])

In [80]:
init_state = tf.keras.layers.Dense(64)(multi_cond_state)
init_state = tf.reshape(init_state, [8, 8])

In [81]:
init_state.shape

TensorShape([8, 8])

In [55]:
init_state=tf.unstack(tf.reshape(init_state, [2, 4]), axis = 0)

In [82]:
forcing_feature_da1 = forcing_feature_da[:, :24*30, :, :, :]
rnn = tf.keras.layers.ConvLSTM2D(filters = 8, kernel_size = 3,
                                               data_format='channels_last',
                                               recurrent_activation='hard_sigmoid',
                                               activation='tanh',
                                               padding='same', return_sequences=True)


In [71]:
forcing_feature_da1.shape

(1, 720, 41, 41, 8)

In [85]:
out = rnn(forcing_feature_da1, initial_state = [init_state, init_state])

IndexError: tuple index out of range

In [28]:
#target
n_sample, n_time_step, nlat, nlon, NUM_CLASSES = target_da.shape

con_target = np.reshape(target_da, (n_sample, n_timestep, -1))

NUM_CELLS1 = 8
NUM_CELLS2 = 4

In [10]:
con_forcing1 = con_forcing[:, :24*30, :]
con_target1 = con_target[:, :24*30, :]

In [60]:
#build a model
class MySimpleModel(tf.keras.Model):
    def __init__(self):
        super(MySimpleModel, self).__init__()
        self.cond = ConditionalRNN(NUM_CELLS1, cell = 'LSTM', dtype = tf.float64, return_sequences = True)
        self.out = tf.keras.layers.ConvLSTM2D(filters = NUM_CELLS2, kernel_size=(3, 3),
                                               data_format='channels_first',
                                               recurrent_activation='hard_sigmoid',
                                               activation='tanh',
                                               padding='same', return_sequences=True)

    def call(self, inputs, **kwargs):
        o = self.cond(inputs)
        o = self.out(o)
        return o


In [57]:
model = MySimpleModel()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

In [58]:
tt = model.cond([con_forcing1] + con_stats)

In [59]:
tt.shape

TensorShape([1, 8])

In [32]:
tt1 = model.out(tt)

ValueError: Input 0 of layer conv_lst_m2d is incompatible with the layer: expected ndim=5, found ndim=3. Full shape received: [1, 720, 8]

In [ ]:
model.call([con_forcing1] + con_stats)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended

In [ ]:
model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
model.fit(x=[con_forcing1] + con_stats, y=con_target1,
         epochs=10)

In [ ]:
model = MySimpleModel()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
model.call([con_forcing] + con_stats)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
model.fit(x=[con_forcing] + con_stats, y=con_target,
          validation_data=([test_inputs, test_targets], con_target),
          epochs=10)

In [ ]:
if 1:
    batch_norm = c.nn.batch_norm
    pooling = c.nn.pooling
    l2 = c.nn.l2
    dr = c.nn.dr
    activation = c.nn.activation

    n_sample, n_timestep, nlat, nlon, n_static_feat = new_static_feature_da.shape
    static_nodes = [int(n_static_feat / 8), 48]
    _, n_timestep, nlat, nlon, nlev_press = prev_press_feature_da.shape
    _, _, nlat, nlon, nlev_forc = forcing_feature_da.shape
    dynamic_nodes = [16, 48]
    n_sample, n_timestep, nlat, nlon, target_number = target_da.shape

    lr = 1e-4
    loss_dict = {
        'mae': 'mae',
        'mse': 'mse',
        'log_loss': log_loss
    }


In [ ]:
if 1:
    filepath = '/glade/scratch/hoangtran/ParFlow-NN/lstm_model.h5'
    t = 1000
    new_model = keras.models.load_model(
        filepath,
        custom_objects={
            "tf": tf,
            "rmse": rmse,
            'log_loss': log_loss,
            "var_ratio": var_ratio,
            "var_loss": var_loss
        }
    )

In [ ]:
new_model

In [ ]:
pred = new_model.predict([
        new_static_feature_da[:, :t, :, :, :],
        prev_press_feature_da[:, :t, :, :, :],
        forcing_feature_da[:, :t, :, :, :]
    ])


In [ ]:
pred.shape

In [ ]:
pred1 = np.reshape(pred,(1,1000,41,41,123))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(pred1[0,100,:,:,100])
plt.colorbar()